In [ ]:
%load_ext autoreload
%load_ext line_profiler

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import kwant 

import functools as ft

import supercurrent
import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

# Define system

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'L_down' :  1000,
               'L_up' : 1000,
               'L_m' : 250,
               'L_x' : 10,
               'a' :  10,
            'mu_from_bottom_of_spin_orbit_bands': True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = .1,
                 alpha_left = .1,
                 alpha_right = .1,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.6,
                 phase = 5.5,
                 T = 0.025)

params = dict(**constants,
              **params_raw)

syst = sns_system.make_wrapped_system(**syst_pars)

# Line profile

In [ ]:
supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-2, max_iterations=1000)

In [ ]:
supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-2, max_iterations=1000)

In [ ]:
supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-2, max_iterations=1000)

In [ ]:
supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-2, max_iterations=1000) / syst_pars['a']

In [ ]:
cut_tag=0
direction=0
tol = 1e-2

(cut_indices, cut_sites) = supercurrent.get_cut_sites_and_indices(syst, cut_tag, direction)
current_operator = kwant.operator.Current(syst,
                                          onsite=supercurrent.sigz,
                                          where=cut_sites)



def f(k):
    I = 0
    p = dict(k_y=k, **params)
    ham = syst.hamiltonian_submatrix(params=p)
    (en, evs) = np.linalg.eigh(ham)

    local_current_operator = current_operator.bind(params=p)
    for (e, ev) in zip(en, evs.T):
        I += supercurrent.fermi_dirac(e.real, p) * local_current_operator(ev)
    return sum(I)*params['e']/params['hbar']

learner = adaptive.IntegratorLearner(f, [0, np.pi], tol)


In [ ]:
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
hv.Points(learner.done_points)

In [ ]:
hv.Points(learner.done_points)

In [ ]:
hv.Points(learner.done_points)

In [ ]:
y, x = np.histogram(list(learner.done_points.keys()), 13)
hv.Points(zip(x,1e-8*y/max(y))) * hv.Points(learner.done_points)

In [ ]:
hv.Points(learner.done_points)

In [ ]:
learner.data

In [ ]:
runner.task.result()